# TranscriptHomies 🧬🤝
#### <span style="color: blue;">_Explore how genes work together. This tool maps gene–gene expression correlations within your dataset, helping you discover coordinated expression patterns and visualize network structures that define biological systems._</span>
#### **Team Members:** Grace Beggs, Caroline Harrer, HeaJin Hong, Tess Kelly, Zilin Xianyu
#### **TAs:** Riley Kellermeyer, Bekah Kim

### Data Input and Organization

### Analysis 

### Output and Visualization